In [8]:
import os
import sys
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.getcwd() + '/database'))
import db_api

sys.path.append(os.path.abspath(os.getcwd() + '/engine'))
from classifiers import *

db = db_api.db_api()

In [9]:
stats = db.get_items('stats')
for stat in stats:
    print stat.name

QUEUE_LEN
ITEMS_PROCESSED
EXEC_TIME
IDLE_TIME
MEMORY_ALLOCATED
NUM_STATES


In [10]:
msus = db.get_items('msus')
for msu in msus:
    print 'ID: ' + str(msu.msu)# + ', TYPE: ' + msu.msu_type.name

ID: 10
ID: 11
ID: 12
ID: 13
ID: 14
ID: 15
ID: 16
ID: 17
ID: 18


In [11]:
ts = db.get_msu_timeseries(msus[0])
for t in ts:
    print t.statistic.name

QUEUE_LEN
ITEMS_PROCESSED
MEMORY_ALLOCATED
NUM_STATES
EXEC_TIME
IDLE_TIME


In [12]:
df = db.get_msu_full_df(msus[0])

X = df.loc[:, ['QUEUE_LEN', 'EXEC_TIME', 'IDLE_TIME']]

k_means_model = do_kmeans(X, 5)

In [15]:
for label in k_means_model.labels_:
    print label

1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
